In [1]:
from PIL import Image
import numpy as np
import os
import cv2

In [2]:
data=[]
labels=[]
size=299
Sol=os.listdir("./Data/Sol/")
for taj in Sol:
    imag=cv2.imread("./Data/Sol/"+taj)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((size, size))
    data.append(np.array(resized_image))
    labels.append(0)

Taj_Mahal=os.listdir("./Data/Taj_Mahal")
for taj in Taj_Mahal:
    imag=cv2.imread("./Data/Taj_Mahal/"+taj)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((size, size))
    data.append(np.array(resized_image))
    labels.append(1)
    
Tour_Effel=os.listdir("./Data/Tour_Effel")
for tour in Tour_Effel:
    imag=cv2.imread("./Data/Tour_Effel/"+tour)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((size, size))
    data.append(np.array(resized_image))
    labels.append(2)
    
Big_Ben=os.listdir("./Data/Big_Ben/")
for tour in Big_Ben:
    imag=cv2.imread("./Data/Big_Ben/"+tour)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((size, size))
    data.append(np.array(resized_image))
    labels.append(3)
Towers=os.listdir("./Data/Towers")
for tour in Towers:
    imag=cv2.imread("./Data/Towers/"+tour)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((size, size))
    data.append(np.array(resized_image))
    labels.append(4)

In [3]:
monuments=np.array(data)
labels=np.array(labels)

In [4]:
s=np.arange(monuments.shape[0])
np.random.shuffle(s)
monuments=monuments[s]
labels=labels[s]

In [5]:
num_classes=len(np.unique(labels))
data_length=len(monuments)
print(len(monuments))
num_classes

2402


5

In [ ]:
x_train=monuments[:(int)(0.7*data_length)]
x_val=monuments[(int)(0.7*data_length):]
train_length=len(x_train)
val_length=len(x_val)
print(len(x_train))
print(len(x_val))

In [ ]:
y_train=labels[:(int)(0.7*data_length)]
y_val=labels[(int)(0.7*data_length):]
print(len(y_train))
print(len(y_val))

In [ ]:
import keras
from keras.utils import np_utils
#One hot encoding
y_train=keras.utils.to_categorical(y_train,num_classes)
y_val=keras.utils.to_categorical(y_val,num_classes)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 32

# Create train generator.
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=.05, 
                                   width_shift_range=.05,
                                   height_shift_range=.05, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train,
                                     batch_size=BATCH_SIZE, seed=1)
                                     
# Create validation generator
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_val, y_val, 
                                   batch_size=BATCH_SIZE, seed=1)

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.applications.xception import Xception
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# Get the InceptionV3 model so we can do transfer learning


base_inception = Xception(weights='imagenet', include_top=False, 
                             input_shape=(299, 299, 3))
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dropout(0.7)(out)
out = Dense(512, activation='relu')(out)
out = Dropout(0.7)(out)
predictions = Dense(num_classes, activation='softmax')(out)

model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
#model.summary()
learning_rate=1e-4
model.compile(optimizer = Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy']) 
batch_size = 32
train_steps_per_epoch = x_train.shape[0] // batch_size
val_steps_per_epoch = x_val.shape[0] // batch_size
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=30)
mc = ModelCheckpoint('X_ception_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# fit model
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=1000, verbose=1,callbacks=[es, mc])

In [ ]:
import matplotlib.pyplot as plt

#get the details form the history object
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
import numpy as np
for i in range(len(val_acc)-30):
    print(str(val_acc[i]))
    